# Capstone Project

## Park Slope Parents Membership
#### Part 2b: EDA

In [2]:
# Load libraries



# import matplotlib.pyplot as plt
# %matplotlib inline
from datetime import datetime, date, timedelta
# from IPython.display import HTML
# from matplotlib import cm as cm
# from mpl_toolkits.mplot3d import Axes3D
# from pandas.io import sql
# from sklearn import metrics
# from sklearn.cluster import DBSCAN
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA 
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics import silhouette_score
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
# from sqlalchemy import create_engine
import csv
# import numpy as np
import pandas as pd 
# import patsy
import plotly.graph_objs as go
import plotly.plotly as py
# from plotly.tools import FigureFactory as FF
from plotly.graph_objs import graph_objs
# import psycopg2 as psy
# import scipy
# import seaborn as sns
# import sklearn
py.sign_in('ajbentley', 'zjfu2vasav')

In [3]:
# read in data

dfn = pd.read_csv("../../projects/psp/raw_data/PSP_data_4capstone.csv")
dfn.info()
dfn.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14914 entries, 0 to 14913
Data columns (total 23 columns):
mem_no                14914 non-null object
address               14914 non-null object
city                  14914 non-null object
state                 14914 non-null object
zip                   14914 non-null int64
joined                14914 non-null object
exp_date              14914 non-null object
status                14914 non-null object
mem_type              14914 non-null object
last_renewal_date     14914 non-null object
gender                14914 non-null object
club_email            14914 non-null object
dup                   14914 non-null object
parent_status         14914 non-null object
kid_count             14914 non-null float64
kid1_bday             14914 non-null object
kid2_bday             14914 non-null object
join_reason           10144 non-null object
advice_grp            14914 non-null int64
classifieds           14914 non-null int64
classifieds_s

,mem_no,address,city,state,zip,joined,exp_date,status,mem_type,last_renewal_date,...,parent_status,kid_count,kid1_bday,kid2_bday,join_reason,advice_grp,classifieds,classifieds_spouse,tony_kids,discovered
0,00002,438_12th_street,brooklyn,NY,11215,2009-02-16,2020-02-15,Active,Lifetime Member,2009-02-16,...,Yes,2.0,4/14/85,4/14/85,dfs,1,1,1,Yes,A PSP member who is a friend/neighbor
1,00004,580_5th_street,brooklyn,NY,11215,2009-04-13,2020-04-12,Active,Lifetime Member,2009-04-13,...,Yes,2.0,12/11/02,12/11/02,NaN,1,1,0,Yes,A PSP member I don't know told me about it
2,00101,502_13th_st,brooklyn,NY,11215,2002-07-17,2014-09-15,Expired,1 year membership ($40),2002-07-17,...,Yes,2.0,1/1/01,1/1/01,no,1,1,0,No,Other
3,00118,1512_10th_ave,brooklyn,NY,11215,2002-08-13,2017-06-15,Active,1 year membership ($40),2002-08-13,...,Yes,2.0,5/17/02,9/14/04,This is a renewal,1,1,0,No,A PSP member who is a friend/neighbor
4,00121,434_13th_st,brooklyn,NY,11215,2002-08-26,2019-06-16,Active,3 year membership ($110),2002-08-26,...,Yes,2.0,10/5/01,7/31/07,"Yes, I'm in love with you, Susan Fox! :-)",1,1,0,Yes,A PSP member who is a friend/neighbor


In [4]:
# # not all types carried over from when the data was saved (datetime in particular, which
# # pleases me not at all).

dfn.joined = pd.to_datetime(dfn.joined, format='%Y/%m/%d')
dfn.exp_date = pd.to_datetime(dfn.exp_date, format='%Y/%m/%d')
dfn.last_renewal_date = pd.to_datetime(dfn.last_renewal_date, format='%Y/%m/%d')
dfn.kid1_bday = pd.to_datetime(dfn.kid1_bday, format='%m/%d/%y')
dfn.kid2_bday = pd.to_datetime(dfn.kid2_bday, format='%m/%d/%y')

In [5]:
# a few changes off the bat

# setting member number as index
dfn.set_index('mem_no')

# getting rid of a few columns

dfn.drop(["address",'join_reason'], axis=1, inplace=True)

I discovered that in 2007 PSP got a website and in 2009 began charging for membership. After exploring year to year data I concluded that these changes were disruptive / significant and chose to cut data prior to 2010.

In order to cut those dates without losing members who joined prior to 2010 I'll first dummy out columns representing membership in each year and will then drop rows that don't indicate membership in 2010 or sooner.

In [6]:
# make a few dummies
# patsy would be easier--relearn

status_dummy = pd.get_dummies(dfn['status'], prefix='status')
memtype_dummy = pd.get_dummies(dfn['mem_type'], prefix='mem_type')
email_dummy = pd.get_dummies(dfn['club_email'], prefix='club_email')
dup_dummy = pd.get_dummies(dfn['dup'], prefix='dup')
parent_dummy = pd.get_dummies(dfn['parent_status'], prefix='parent_status')

advice_dummy = pd.get_dummies(dfn['advice_grp'], prefix='advice_grp')
classifieds_dummy = pd.get_dummies(dfn['classifieds'], prefix='classifieds')
class_sp_dummy = pd.get_dummies(dfn['classifieds_spouse'], prefix='classifieds_spouse')
tony_dummy = pd.get_dummies(dfn['tony_kids'], prefix='tony_dids')
disc_dummy = pd.get_dummies(dfn['discovered'], prefix='discovered')


dfn = dfn.join(status_dummy)
dfn = dfn.join(memtype_dummy)
dfn = dfn.join(email_dummy)
dfn = dfn.join(dup_dummy)
dfn = dfn.join(parent_dummy)
dfn = dfn.join(advice_dummy)
dfn = dfn.join(classifieds_dummy)
dfn = dfn.join(class_sp_dummy)
dfn = dfn.join(tony_dummy)
dfn = dfn.join(disc_dummy)

In [7]:
# columns for joined month and year

dfn['join_year'] = dfn['joined'].dt.year
dfn['join_month'] = dfn['joined'].dt.month

# columns for exp_date year

dfn['exp_year'] = dfn['exp_date'].dt.year

# # columns for 1st kid's birth month and year
dfn['k1bday_year'] = dfn['kid1_bday'].dt.year
dfn['k1bday_month'] = dfn['kid1_bday'].dt.month

# # columns for 2nd kid's birth month and year
dfn['k2bday_year'] = dfn['kid2_bday'].dt.year
dfn['k2bday_month'] = dfn['kid2_bday'].dt.month



dfn.head()

,mem_no,city,state,zip,joined,exp_date,status,mem_type,last_renewal_date,gender,...,"discovered_Heard about it through a magazine, newspaper, blog",discovered_I don't remember,discovered_Other,join_year,join_month,exp_year,k1bday_year,k1bday_month,k2bday_year,k2bday_month
0,00002,brooklyn,NY,11215,2009-02-16,2020-02-15,Active,Lifetime Member,2009-02-16,Female,...,0.0,0.0,0.0,2009,2,2020,1985,4,1985,4
1,00004,brooklyn,NY,11215,2009-04-13,2020-04-12,Active,Lifetime Member,2009-04-13,Female,...,0.0,0.0,0.0,2009,4,2020,2002,12,2002,12
2,00101,brooklyn,NY,11215,2002-07-17,2014-09-15,Expired,1 year membership ($40),2002-07-17,Female,...,0.0,0.0,1.0,2002,7,2014,2001,1,2001,1
3,00118,brooklyn,NY,11215,2002-08-13,2017-06-15,Active,1 year membership ($40),2002-08-13,Female,...,0.0,0.0,0.0,2002,8,2017,2002,5,2004,9
4,00121,brooklyn,NY,11215,2002-08-26,2019-06-16,Active,3 year membership ($110),2002-08-26,Female,...,0.0,0.0,0.0,2002,8,2019,2001,10,2007,7


In [8]:
# check dates for out of consideration range. basically for grandparents who are using their children's birth dates,
# not their grandchildren's. Org started in 2002 so will assume anything prior to 1990 will be out of range

dfn = pd.DataFrame(dfn.loc[dfn['k1bday_year'] >= 1995])
dfn = pd.DataFrame(dfn.loc[dfn['k2bday_year'] >= 1995])
dfn = pd.DataFrame(dfn.loc[dfn['k1bday_year'] < 2018])
dfn = pd.DataFrame(dfn.loc[dfn['k2bday_year'] < 2018])

In [9]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14591 entries, 1 to 14913
Data columns (total 63 columns):
mem_no                                                                            14591 non-null object
city                                                                              14591 non-null object
state                                                                             14591 non-null object
zip                                                                               14591 non-null int64
joined                                                                            14591 non-null datetime64[ns]
exp_date                                                                          14591 non-null datetime64[ns]
status                                                                            14591 non-null object
mem_type                                                                          14591 non-null object
last_renewal_date                           

In [10]:
# enumerate some columns that had been dummied so that they come up on histograms
# patsy would speed this up

dfn['mem_type'].replace('1 year membership ($40)', 0, inplace=True)
dfn['mem_type'].replace('2 Year Membership ($75)', 1, inplace=True)
dfn['mem_type'].replace('3 year membership ($110)', 2, inplace=True)
dfn['mem_type'].replace('5 year membership ($175)', 3, inplace=True)
dfn['mem_type'].replace('Complimentary', 4, inplace=True)
dfn['mem_type'].replace('Lifetime Member', 5, inplace=True)
dfn['mem_type'].replace('Trial Membership', 6, inplace=True)

dfn['parent_status'].replace('No', 0, inplace=True)
dfn['parent_status'].replace('No, but we are pregnant/adopting', 1, inplace=True)
dfn['parent_status'].replace('Yes', 2, inplace=True)

dfn['discovered'].replace('A PSP member I don\'t know told me about it', 0, inplace=True)
dfn['discovered'].replace('A PSP member who is a friend/neighbor', 1, inplace=True)
dfn['discovered'].replace('Found it through Yahoo', 2, inplace=True)
dfn['discovered'].replace('Found it through a Google search', 3, inplace=True)
dfn['discovered'].replace('Heard about it on another online parenting group (Urban Baby, etc.)', 4, inplace=True)
dfn['discovered'].replace('Heard about it through a magazine, newspaper, blog', 5, inplace=True)
dfn['discovered'].replace('I don\'t remember', 6, inplace=True)
dfn['discovered'].replace('NA', 5, inplace=True)
dfn['discovered'].replace('Other', 6, inplace=True)



In [11]:
# historam of month joined
data = [go.Histogram(x=dfn.join_month)]
py.iplot(data, thickness=5)

# Members are more likely to join in Summer (June / July) than at any other time of year.
# Following July, enrollment steps down through the rest of the year with December being the softest month.
# January is the 3rd strongest month, though that might be a function of December being the lowest.

In [12]:
# histogram of number of children
# data = [go.Histogram(x=dfn.kid_count)]
# py.iplot(data)

# Most members have had only one child, but the number of two children families has been very strong as well.
# I had been toying with the idea of just using 1-child families since that would be easier for some comparisons,
# but that is clearly not an option here. The good news is that there are very few 3+ families so the fact that we
# don't have birth dates past #2 is less of an issue.

# I do have to ask SF if members are asked to update the number of children they have after a child has been born as
# This would skew the results.

# Please note that the original distribution for this feature included a "more than four" category. There were few
# enough members of that category that I merged them with the 4 (to make 4+)

In [13]:
# Compare birth patterns by month for first and second child

# x0 = dfn.k1bday_month
# x1 = dfn.k2bday_month

# trace1 = go.Histogram(
#     x=x0,
#     opacity=0.75
# )
# trace2 = go.Histogram(
#     x=x1,
#     opacity=0.75
# )
# data = [trace1, trace2]
# layout = go.Layout(
#     barmode='overlay'
# )
# fig = go.Figure(data=data, layout=layout)
# py.iplot(fig)

# Birth patterns are very similar, never more than 50 children between number born in each month 

In [14]:
# histogram of how discovered
# data = [go.Histogram(x=dfn.discovered)]
# py.iplot(data)


# Far and away the top means of finding PSP was through a friend/neighbor who is a member of the group. 

In [15]:
# histogram of membership type
# data = [go.Histogram(x=dfn.mem_type)]
# py.iplot(data)


# Far and away the top membership option is annual.

In [16]:
dfn.T.head(60)

,1,2,3,4,5,6,7,8,9,10,...,14904,14905,14906,14907,14908,14909,14910,14911,14912,14913
mem_no,00004,00101,00118,00121,00122,00124,00127,00139,00148,00159,...,35083,35084,35085,35086,35087,35088,35089,35090,35091,35092
city,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,...,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn,brooklyn
state,NY,NY,NY,NY,NY,NY,NY,NY,NY,NY,...,NY,NY,NY,NY,NY,NY,NY,NY,NY,NY
zip,11215,11215,11215,11215,11215,11215,11217,11215,11215,11226,...,11215,11217,11232,11215,11203,11215,11238,11238,11238,11215
joined,2009-04-13 00:00:00,2002-07-17 00:00:00,2002-08-13 00:00:00,2002-08-26 00:00:00,2002-08-28 00:00:00,2002-08-29 00:00:00,2002-09-03 00:00:00,2002-09-18 00:00:00,2002-10-03 00:00:00,2002-10-15 00:00:00,...,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-15 00:00:00,2016-11-15 00:00:00
exp_date,2020-04-12 00:00:00,2014-09-15 00:00:00,2017-06-15 00:00:00,2019-06-16 00:00:00,2017-06-20 00:00:00,2017-06-18 00:00:00,2017-05-31 00:00:00,2018-05-15 00:00:00,2012-05-03 00:00:00,2014-03-24 00:00:00,...,2017-11-13 00:00:00,2017-11-13 00:00:00,2017-11-13 00:00:00,2017-11-13 00:00:00,2017-11-13 00:00:00,2017-11-14 00:00:00,2017-11-13 00:00:00,2017-11-13 00:00:00,2017-11-14 00:00:00,2017-11-14 00:00:00
status,Active,Expired,Active,Active,Active,Active,Active,Active,Expired,Expired,...,Pending,Pending,Prospective,Pending,Prospective,Active,Pending,Pending,Prospective,Prospective
mem_type,5,0,0,2,0,1,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
last_renewal_date,2009-04-13 00:00:00,2002-07-17 00:00:00,2002-08-13 00:00:00,2002-08-26 00:00:00,2002-08-28 00:00:00,2002-08-29 00:00:00,2002-09-03 00:00:00,2002-09-18 00:00:00,2002-10-03 00:00:00,2002-10-15 00:00:00,...,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-14 00:00:00,2016-11-15 00:00:00,2016-11-15 00:00:00
gender,Female,Female,Female,Female,Female,Female,Female,Female,Female,Female,...,Female,Female,Female,Female,Female,Female,Male,Male,Female,Female


In [17]:
pd.options.display.max_columns = 999
dfn.describe()

/Users/EKandTower/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning:

Invalid value encountered in percentile



,zip,mem_type,parent_status,kid_count,advice_grp,classifieds,classifieds_spouse,discovered,status_Active,status_Expired,status_Frozen,status_Pending,status_Prospective,mem_type_1 year membership ($40),mem_type_2 Year Membership ($75),mem_type_3 year membership ($110),mem_type_5 year membership ($175),mem_type_Complimentary,mem_type_Lifetime Member,mem_type_Trial Membership,club_email_No,club_email_Yes,dup_NO,dup_YES,parent_status_No,"parent_status_No, but we are pregnant/adopting",parent_status_Yes,advice_grp_0,advice_grp_1,classifieds_0,classifieds_1,classifieds_spouse_0,classifieds_spouse_1,tony_dids_No,tony_dids_Yes,discovered_A PSP member I don't know told me about it,discovered_A PSP member who is a friend/neighbor,discovered_Found it through Yahoo,discovered_Found it through a Google search,"discovered_Heard about it on another online parenting group (Urban Baby, etc.)","discovered_Heard about it through a magazine, newspaper, blog",discovered_I don't remember,discovered_Other,join_year,join_month,exp_year,k1bday_year,k1bday_month,k2bday_year,k2bday_month
count,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14589.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000
mean,11559.493866,0.154547,1.858406,1.359091,0.853334,0.902543,0.023439,2.118171,0.310739,0.681447,0.005757,0.001645,0.000411,0.887191,0.084230,0.019601,0.006511,0.000822,0.001576,0.000069,0.177233,0.822767,0.645261,0.354739,0.004318,0.132959,0.862724,0.146666,0.853334,0.097457,0.902543,0.976561,0.023439,0.338359,0.661641,0.044959,0.645124,0.010006,0.092797,0.027071,0.013364,0.097389,0.069152,2011.916113,6.356727,2014.737578,2009.794257,6.591118,2010.822973,6.578028
std,5324.033728,0.502393,0.360818,0.693542,0.353785,0.296590,0.151299,1.948794,0.462812,0.465931,0.075659,0.040525,0.020275,0.316370,0.277742,0.138630,0.080430,0.028667,0.039673,0.008279,0.381878,0.381878,0.478451,0.478451,0.065570,0.339542,0.344151,0.353785,0.353785,0.296590,0.296590,0.151299,0.151299,0.473168,0.473168,0.207222,0.478492,0.099533,0.290158,0.162297,0.114833,0.296497,0.253722,2.676137,3.277253,2.302012,4.415239,3.424408,3.939082,3.414689
min,1002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2002.000000,1.000000,2009.000000,1995.000000,1.000000,1995.000000,1.000000
25%,11215.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010.000000,4.000000,2013.000000,2007.000000,4.000000,2009.000000,4.000000
50%,11215.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,NaN,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2012.000000,6.

In [18]:
dfn['mem_duration'] = dfn['exp_year'] - dfn['join_year']

dfn.describe()

,zip,mem_type,parent_status,kid_count,advice_grp,classifieds,classifieds_spouse,discovered,status_Active,status_Expired,status_Frozen,status_Pending,status_Prospective,mem_type_1 year membership ($40),mem_type_2 Year Membership ($75),mem_type_3 year membership ($110),mem_type_5 year membership ($175),mem_type_Complimentary,mem_type_Lifetime Member,mem_type_Trial Membership,club_email_No,club_email_Yes,dup_NO,dup_YES,parent_status_No,"parent_status_No, but we are pregnant/adopting",parent_status_Yes,advice_grp_0,advice_grp_1,classifieds_0,classifieds_1,classifieds_spouse_0,classifieds_spouse_1,tony_dids_No,tony_dids_Yes,discovered_A PSP member I don't know told me about it,discovered_A PSP member who is a friend/neighbor,discovered_Found it through Yahoo,discovered_Found it through a Google search,"discovered_Heard about it on another online parenting group (Urban Baby, etc.)","discovered_Heard about it through a magazine, newspaper, blog",discovered_I don't remember,discovered_Other,join_year,join_month,exp_year,k1bday_year,k1bday_month,k2bday_year,k2bday_month,mem_dur_detailed
count,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14589.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591
mean,11559.493866,0.154547,1.858406,1.359091,0.853334,0.902543,0.023439,2.118171,0.310739,0.681447,0.005757,0.001645,0.000411,0.887191,0.084230,0.019601,0.006511,0.000822,0.001576,0.000069,0.177233,0.822767,0.645261,0.354739,0.004318,0.132959,0.862724,0.146666,0.853334,0.097457,0.902543,0.976561,0.023439,0.338359,0.661641,0.044959,0.645124,0.010006,0.092797,0.027071,0.013364,0.097389,0.069152,2011.916113,6.356727,2014.737578,2009.794257,6.591118,2010.822973,6.578028,1034 days 09:33:59.202247
std,5324.033728,0.502393,0.360818,0.693542,0.353785,0.296590,0.151299,1.948794,0.462812,0.465931,0.075659,0.040525,0.020275,0.316370,0.277742,0.138630,0.080430,0.028667,0.039673,0.008279,0.381878,0.381878,0.478451,0.478451,0.065570,0.339542,0.344151,0.353785,0.353785,0.296590,0.296590,0.151299,0.151299,0.473168,0.473168,0.207222,0.478492,0.099533,0.290158,0.162297,0.114833,0.296497,0.253722,2.676137,3.277253,2.302012,4.415239,3.424408,3.939082,3.414689,929 days 00:16:52.038733
min,1002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2002.000000,1.000000,2009.000000,1995.000000,1.000000,1995.000000,1.000000,1 days 00:00:00
25%,11215.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010.000000,4.000000,2013.000000,2007.000000,4.000000,2009.000000,4.000000,364 days 00:00:00
50%,11215.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,NaN,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000

In [19]:
# histogram of membership length
# data = [go.Histogram(x=dfn.mem_duration)]
# py.iplot(data)

# Most people have only had 1-year memberships

In [20]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14591 entries, 1 to 14913
Data columns (total 64 columns):
mem_no                                                                            14591 non-null object
city                                                                              14591 non-null object
state                                                                             14591 non-null object
zip                                                                               14591 non-null int64
joined                                                                            14591 non-null datetime64[ns]
exp_date                                                                          14591 non-null datetime64[ns]
status                                                                            14591 non-null object
mem_type                                                                          14591 non-null int64
last_renewal_date                            

In [21]:
# creating a column that shows the difference between when people join and when their kids are born

dfn['join_v_birth'] = (dfn['joined']-dfn['kid1_bday']).astype('timedelta64[m]')


In [22]:
# for whatever reason, the numbers that come out of a timedelta on months needs to be divided by 1440 in order to get
# to the number of days, which can then be converted to months.

k = ((sum(dfn.join_v_birth) / len(dfn.join_v_birth)))
print k
print k/43200
print k/1440
print ((k/1440)/30)

1105368.21876
25.5872272862
767.616818587
25.5872272862


In [23]:
# for some reason there are still some strange numbers in here, including people joining the organization when their
# children are in their 20s. In order to exclude outliers I'm going to create a more focused dataframe that will be 
# more useful for the analysis

dfk = pd.DataFrame(dfn.loc[dfn['join_v_birth'] < 6220800])
dfk = pd.DataFrame(dfk.loc[dfk['join_v_birth'] > -1036800])
dfk.head()

,mem_no,city,state,zip,joined,exp_date,status,mem_type,last_renewal_date,gender,club_email,dup,parent_status,kid_count,kid1_bday,kid2_bday,advice_grp,classifieds,classifieds_spouse,tony_kids,discovered,status_Active,status_Expired,status_Frozen,status_Pending,status_Prospective,mem_type_1 year membership ($40),mem_type_2 Year Membership ($75),mem_type_3 year membership ($110),mem_type_5 year membership ($175),mem_type_Complimentary,mem_type_Lifetime Member,mem_type_Trial Membership,club_email_No,club_email_Yes,dup_NO,dup_YES,parent_status_No,"parent_status_No, but we are pregnant/adopting",parent_status_Yes,advice_grp_0,advice_grp_1,classifieds_0,classifieds_1,classifieds_spouse_0,classifieds_spouse_1,tony_dids_No,tony_dids_Yes,discovered_A PSP member I don't know told me about it,discovered_A PSP member who is a friend/neighbor,discovered_Found it through Yahoo,discovered_Found it through a Google search,"discovered_Heard about it on another online parenting group (Urban Baby, etc.)","discovered_Heard about it through a magazine, newspaper, blog",discovered_I don't remember,discovered_Other,join_year,join_month,exp_year,k1bday_year,k1bday_month,k2bday_year,k2bday_month,mem_dur_detailed,join_v_birth
1,00004,brooklyn,NY,11215,2009-04-13,2020-04-12,Active,5,2009-04-13,Female,Yes,NO,2,2.0,2002-12-11,2002-12-11,1,1,0,Yes,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009,4,2020,2002,12,2002,12,4017 days,3333600.0
2,00101,brooklyn,NY,11215,2002-07-17,2014-09-15,Expired,0,2002-07-17,Female,Yes,YES,2,2.0,2001-01-01,2001-01-01,1,1,0,No,6.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2002,7,2014,2001,1,2001,1,4443 days,809280.0
3,00118,brooklyn,NY,11215,2002-08-13,2017-06-15,Active,0,2002-08-13,Female,Yes,NO,2,2.0,2002-05-17,2004-09-14,1,1,0,No,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2002,8,2017,2002,5,2004,9,5420 days,126720.0
4,00121,brooklyn,NY,11215,2002-08-26,2019-06-16,Active,2,2002-08-26,Female,Yes,YES,2,2.0,2001-10-05,2007-07-31,1,1,0,Yes,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2002,8,2019,2001,10,2007,7,6138 days,468000.0
5,00122,brooklyn,NY,11215,2002-08-28,2017-06-20,Active,0,2002-08-28,Female,Yes,YES,2,1.0,2002-07-18,2002-07-18,1,1,0,No,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2002,8,2017,2002,7,2002,7,5410 days,59040.0


In [24]:
dfn.describe()

,zip,mem_type,parent_status,kid_count,advice_grp,classifieds,classifieds_spouse,discovered,status_Active,status_Expired,status_Frozen,status_Pending,status_Prospective,mem_type_1 year membership ($40),mem_type_2 Year Membership ($75),mem_type_3 year membership ($110),mem_type_5 year membership ($175),mem_type_Complimentary,mem_type_Lifetime Member,mem_type_Trial Membership,club_email_No,club_email_Yes,dup_NO,dup_YES,parent_status_No,"parent_status_No, but we are pregnant/adopting",parent_status_Yes,advice_grp_0,advice_grp_1,classifieds_0,classifieds_1,classifieds_spouse_0,classifieds_spouse_1,tony_dids_No,tony_dids_Yes,discovered_A PSP member I don't know told me about it,discovered_A PSP member who is a friend/neighbor,discovered_Found it through Yahoo,discovered_Found it through a Google search,"discovered_Heard about it on another online parenting group (Urban Baby, etc.)","discovered_Heard about it through a magazine, newspaper, blog",discovered_I don't remember,discovered_Other,join_year,join_month,exp_year,k1bday_year,k1bday_month,k2bday_year,k2bday_month,mem_dur_detailed,join_v_birth
count,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14589.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591.000000,14591,1.459100e+04
mean,11559.493866,0.154547,1.858406,1.359091,0.853334,0.902543,0.023439,2.118171,0.310739,0.681447,0.005757,0.001645,0.000411,0.887191,0.084230,0.019601,0.006511,0.000822,0.001576,0.000069,0.177233,0.822767,0.645261,0.354739,0.004318,0.132959,0.862724,0.146666,0.853334,0.097457,0.902543,0.976561,0.023439,0.338359,0.661641,0.044959,0.645124,0.010006,0.092797,0.027071,0.013364,0.097389,0.069152,2011.916113,6.356727,2014.737578,2009.794257,6.591118,2010.822973,6.578028,1034 days 09:33:59.202247,1.105368e+06
std,5324.033728,0.502393,0.360818,0.693542,0.353785,0.296590,0.151299,1.948794,0.462812,0.465931,0.075659,0.040525,0.020275,0.316370,0.277742,0.138630,0.080430,0.028667,0.039673,0.008279,0.381878,0.381878,0.478451,0.478451,0.065570,0.339542,0.344151,0.353785,0.353785,0.296590,0.296590,0.151299,0.151299,0.473168,0.473168,0.207222,0.478492,0.099533,0.290158,0.162297,0.114833,0.296497,0.253722,2.676137,3.277253,2.302012,4.415239,3.424408,3.939082,3.414689,929 days 00:16:52.038733,1.828767e+06
min,1002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2002.000000,1.000000,2009.000000,1995.000000,1.000000,1995.000000,1.000000,1 days 00:00:00,-4.341600e+06
25%,11215.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010.000000,4.000000,2013.000000,2007.000000,4.000000,2009.000000,4.000000,364 days 00:00:00,-8.928000e+04
50%,11215.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,NaN,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0000

In [25]:
# histogram of difference between PSP join and 1st child's birth
data = [go.Histogram(x=(dfk.join_v_birth/43200))]
py.iplot(data)

# Reminder: this chart excludes any record that showed a join date over 12 years after the first child's birth or
# over 2 years before the first child's birth.

# On average (median) parents join PSP about 5 months after their first child is born (includes those who join pre).

In [26]:
# separating out positives and negatives in join v birth (those who joined pre and post birth)
jvb_p = []
jvb_n = []
jvb_z = []

for n in dfk.join_v_birth:
    if n > 0:
        jvb_p.append(n)
    elif n < 0:
        jvb_n.append(n)
    else:
        jvb_z.append(n)
            

In [27]:
# about 2/3 of members join after their child is born.

pl = len(jvb_p)
nl = len(jvb_n)
zl = len(jvb_z)

lensum = (pl+nl+zl)
print pl, nl, zl 
print lensum

print pl/14114.
print nl/14114.

df_jvb_len = pd.DataFrame([pl, nl, zl])

9377 4728 9
14114
0.664375797081
0.334986538189


In [28]:
jvb_pos = pd.Series(jvb_p)
jvb_neg = pd.Series(jvb_n)
jvb_zed = pd.Series(jvb_z)

In [29]:
jvb_pos.describe()

# among members who join after their first child is born, on average (median) their child is 22 months old. 
# the 1st quartile (lowest 25% of join v birth) joined when their child was 4.5 months old. 

count    9.377000e+03
mean     1.507524e+06
std      1.554661e+06
min      1.440000e+03
25%      1.944000e+05
50%      9.532800e+05
75%      2.412000e+06
max      6.217920e+06
dtype: float64

In [30]:
jvb_neg.describe()

# among members who join before their first child is born, on average (median) their child is due in 3.6 months. 
# the 1st quartile (lowest 25% of join v birth) joined when their child was due in 5.3 months. 
# This is a much tighter cluster than for those who join after they have their child.

count    4.728000e+03
mean    -1.713959e+05
std      1.223827e+05
min     -1.035360e+06
25%     -2.304000e+05
50%     -1.569600e+05
75%     -9.072000e+04
max     -1.440000e+03
dtype: float64

In [31]:
# creating a new column which shows the difference between the member's
# expiration date and the 2nd child's birthday (defaults to 1st child if no second)

dfn['exp_v_birth'] = (dfn['exp_date']-dfn['kid2_bday']).astype('timedelta64[m]')


In [32]:
# largest membership length before "lifetime" is 5 years so I will exclude any instances of a membership longer than
# 5 years. Also excluding records with negative numbers here.

dfk2 = pd.DataFrame(dfn.loc[dfn['exp_v_birth'] <= 2628000])
dfk2 = pd.DataFrame(dfk2.loc[dfk2['exp_v_birth'] >= 0])

In [33]:
# breaking out members who have lapsed from members who are still active

df_exp = pd.DataFrame(dfk2.loc[dfk2['status'] == 'Expired'])

In [34]:
df_exp.describe()

,zip,mem_type,parent_status,kid_count,advice_grp,classifieds,classifieds_spouse,discovered,status_Active,status_Expired,status_Frozen,status_Pending,status_Prospective,mem_type_1 year membership ($40),mem_type_2 Year Membership ($75),mem_type_3 year membership ($110),mem_type_5 year membership ($175),mem_type_Complimentary,mem_type_Lifetime Member,mem_type_Trial Membership,club_email_No,club_email_Yes,dup_NO,dup_YES,parent_status_No,"parent_status_No, but we are pregnant/adopting",parent_status_Yes,advice_grp_0,advice_grp_1,classifieds_0,classifieds_1,classifieds_spouse_0,classifieds_spouse_1,tony_dids_No,tony_dids_Yes,discovered_A PSP member I don't know told me about it,discovered_A PSP member who is a friend/neighbor,discovered_Found it through Yahoo,discovered_Found it through a Google search,"discovered_Heard about it on another online parenting group (Urban Baby, etc.)","discovered_Heard about it through a magazine, newspaper, blog",discovered_I don't remember,discovered_Other,join_year,join_month,exp_year,k1bday_year,k1bday_month,k2bday_year,k2bday_month,mem_dur_detailed,join_v_birth,exp_v_birth
count,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7456.000000,7457.0,7457.0,7457.0,7457.0,7457.0,7457.000000,7457.000000,7457.000000,7457.0,7457.000000,7457.0,7457.0,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457.000000,7457,7.457000e+03,7.457000e+03
mean,11571.747083,0.031112,1.808770,1.215972,0.847928,0.885611,0.006973,2.047210,0.0,1.0,0.0,0.0,0.0,0.971973,0.025748,0.001877,0.0,0.000402,0.0,0.0,0.215234,0.784766,0.777390,0.222610,0.005230,0.180770,0.814000,0.152072,0.847928,0.114389,0.885611,0.993027,0.006973,0.306155,0.693845,0.045997,0.646641,0.011265,0.106075,0.032319,0.015154,0.082741,0.059675,2011.709669,6.346654,2013.617809,2010.598900,6.630817,2011.488266,6.615261,698 days 15:40:14.322113,5.712911e+05,1.110433e+06
std,5465.814591,0.196803,0.406378,0.613677,0.359115,0.318305,0.083220,1.865403,0.0,0.0,0.0,0.0,0.0,0.165062,0.158392,0.043292,0.0,0.020055,0.0,0.0,0.411012,0.411012,0.416026,0.416026,0.072134,0.384853,0.389133,0.359115,0.359115,0.318305,0.318305,0.083220,0.083220,0.460926,0.460926,0.209493,0.478045,0.105542,0.307954,0.176857,0.122172,0.275509,0.236900,2.155578,3.292453,1.852847,2.917024,3.420358,2.280940,3.419232,514 days 01:10:30.311986,1.069228e+06,6.497555e+05
min,1060.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2003.000000,1.000000,2010.000000,1995.000000,1.000000,2005.000000,1.000000,3 days 00:00:00,-4.341600e+06,5.760000e+03
25%,11215.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,NaN,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010.000000,4.000000,2012.000000,2009.000000,4.000000,2010.000000,4.000000,364 days 00:00:00,-9.504000e+04,5.688000e+05
50%,11215.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,NaN,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2012.000000,7.000000,2014.000000,2011.000

In [35]:
# histogram of difference between PSP expiration date and 2nd child's arrival for expired members
data = [go.Histogram(x=(df_exp.exp_v_birth/43200))]
py.iplot(data)

# this is a much more evenly distributed dataset, though still right-skewed.

# On average (median) PSP memberships have lapsed about 22 months after their second child is born (first child if
# only).

# This is a VERY interesting number as it is also the median age at which new members join (join date versus 1st
# child's birth. This suggests there may be a "ships in the night" pattern.

In [36]:


# # give the clusters nicer names

# def new_name(x):
#     if x == 0:
#         return "Cluster A"
#     elif x == 1:
#         return "Cluster B"
#     else:
#         return "Cluster C"
    
# dfc['cluster_name'] = dfc['cluster'].apply(new_name);

In [37]:
# dfn.to_csv("../../projects/psp/raw_data/psp_numerical.csv", index=False)

